In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

# external imports
import pandas as pd
import plaid
from google.cloud import bigquery

In [2]:
print(sys.path)

['../src', '../', '/Users/zachcox/repos/finance/personal_finance/notebooks', '/Users/zachcox/.pyenv/versions/3.8.2/lib/python38.zip', '/Users/zachcox/.pyenv/versions/3.8.2/lib/python3.8', '/Users/zachcox/.pyenv/versions/3.8.2/lib/python3.8/lib-dynload', '', '/Users/zachcox/.pyenv/versions/3.8.2/envs/personal_finance/lib/python3.8/site-packages']


In [2]:
# src imports
from src.utils.bq_utils import BqUtils
from src.utils.plaid_utils import PlaidUtils

In [4]:
from src.my_functions import *

In [4]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)

In [5]:
PLAID_CLIENT_ID = "65975384ab670e001c0aaf0d"
# PLAID_SECRET="56e33c77237c8c9e45f5c066b8b2fa" #production
PLAID_SECRET = "9294dd5ca4a5c99d90da56640f40e5"  # sandbox
PLAID_ENV = "sandbox"
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
PLAID_HOST = plaid.Environment.Sandbox
# PLAID_REDIRECT_URI="https://localhost:3000/"

In [6]:
# Assuming bq_utils_instance is already defined
# bq_client_init = bigquery.Client.from_service_account_json('path/to/your/service_account_key.json')
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)

# Instantiate PlaidUtils with Plaid API credentials
plaid = PlaidUtils(bq_client, client_id=PLAID_CLIENT_ID, client_secret=PLAID_SECRET, host=PLAID_HOST)

In [7]:
bq.get_partition_date()

'20240327'

In [8]:
# cursors_q = """
# SELECT DISTINCT
#     access_token,
#     item_id,
#     "" AS next_cursor
# FROM `zsc-personal.personal_finance.financial_accounts`
# """

# bq.create_table(query=cursors_q, destination_table="zsc-personal.personal_finance.account_cursors_20240325")

In [9]:
cursors_df = plaid.get_latest_cursors(dataset_id="personal_finance", table_prefix="account_cursors_")

In [10]:
chase_access_token = "access-sandbox-747b2b77-8d2f-4318-9c44-635e3fc4575d"
chase_df = cursors_df[cursors_df["access_token"] == chase_access_token]
cursors_df

,access_token,item_id,next_cursor
0,access-sandbox-747b2b77-8d2f-4318-9c44-635e3fc4575d,P1777a93P5HEbJ4kvlNvugd39Xmo1pt7XKLGG,
1,access-sandbox-5f048cf7-c0fe-4811-8769-cb35b8558528,dpd5qZmPm4sGPx1BBnmpfK4DV1bmAPSZ9jggy,


In [11]:
access_token = chase_df["access_token"][0]
item_id = chase_df["item_id"][0]
next_cursor = chase_df["next_cursor"][0]

In [12]:
transactions_df, removed = plaid.get_transactions(access_token=access_token, item_id=item_id, next_cursor=next_cursor)

{
  "added": [
    {
      "account_id": "pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989",
      "account_owner": null,
      "amount": 5.4,
      "authorized_date": "2024-02-23",
      "authorized_datetime": null,
      "category": [
        "Travel",
        "Taxi"
      ],
      "category_id": "22016000",
      "check_number": null,
      "counterparties": [
        {
          "confidence_level": "VERY_HIGH",
          "entity_id": "eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd",
          "logo_url": "https://plaid-merchant-logos.plaid.com/uber_1060.png",
          "name": "Uber",
          "phone_number": null,
          "type": "merchant",
          "website": "uber.com"
        }
      ],
      "date": "2024-02-24",
      "datetime": null,
      "iso_currency_code": "USD",
      "location": {
        "address": null,
        "city": null,
        "country": null,
        "lat": null,
        "lon": null,
        "postal_code": null,
        "region": null,
        "store_number": null
      },


In [13]:
transactions_df

,account_id,account_owner,status,amount,authorized_date,authorized_datetime,category,category_id,check_number,counterparties,date,datetimes,currency_code,address,city,region,postal_code,latitude,longitude,merchant_entity_id,merchant_name,name,payment_channel,reference_number,ppd_id,payee,by_order_of,payer,payment_method,payment_processor,reason,pending,pending_transaction_id,personal_finance_category_confidence_level,personal_finance_category_detailed,personal_finance_category_primary,transaction_code,transaction_id,transaction_type,unofficial_currency_code,website
0,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,5.40,2024-02-23,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd', 'phone_number': None}]",2024-02-24,None,USD,None,None,None,None,None,None,eyg8o776k0QmNgVpAmaQj4WgzW9Qzo6O51gdd,Uber,Uber 063015 SF**POOL**,online,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRANSPORTATION_TAXIS_AND_RIDE_SHARES,TRANSPORTATION,None,j4777q1yDlcM95ElByVBCDDvGnRWJWfveWQJK,special,None,uber.com
1,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,-500.00,2024-02-22,None,"[Travel, Airlines and Aviation Services]",22001000,None,"[{'name': 'United Airlines', 'type': 'merchant', 'website': 'united.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/united_airlines_1065.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4', 'phone_number': None}]",2024-02-22,None,USD,None,None,None,None,None,None,NKDjqyAdQQzpyeD8qpLnX0D6yvLe2KYKYYzQ4,United Airlines,United Airlines,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,TRAVEL_FLIGHTS,TRAVEL,None,7nXXXMdZGEuQvZ3jmXGmCyydazKx3xs8qJwXy,special,None,united.com
2,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,12.00,2024-02-21,None,"[Food and Drink, Restaurants, Fast Food]",13005032,None,"[{'name': 'McDonald's', 'type': 'merchant', 'website': 'mcdonalds.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/mcdonalds_619.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22', 'phone_number': None}]",2024-02-21,None,USD,None,None,None,None,None,None,vzWXDWBjB06j5BJoD3Jo84OJZg7JJzmqOZA22,McDonald's,McDonald's,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_FAST_FOOD,FOOD_AND_DRINK,None,eBwwwk7A95C3Gyegk51kCZZxrvMXVXiaG9xW9,place,None,mcdonalds.com
3,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,4.33,2024-02-21,None,"[Food and Drink, Restaurants, Coffee Shop]",13005043,None,"[{'name': 'Starbucks', 'type': 'merchant', 'website': 'starbucks.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/starbucks_956.png', 'confidence_level': 'VERY_HIGH', 'entity_id': 'NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb', 'phone_number': None}]",2024-02-21,None,USD,None,None,None,None,None,None,NZAJQ5wYdo1W1p39X5q5gpb15OMe39pj4pJBb,Starbucks,Starbucks,in store,None,None,None,None,None,None,None,None,False,None,VERY_HIGH,FOOD_AND_DRINK_COFFEE,FOOD_AND_DRINK,None,QLZZZ4vxj5tQedKM8qN8C77X3Bmj6jTGVmjKQ,place,None,starbucks.com
4,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,89.40,2024-02-19,None,"[Food and Drink, Restaurants]",13005000,None,"[{'name': 'FUN', 'type': 'merchant', 'website': None, 'logo_url': None, 'confidence_level': 'LOW', 'entity_id': None, 'phone_number': None}]",2024-02-20,None,USD,None,None,None,None,None,None,None,FUN,SparkFun,in store,None,None,None,None,None,None,None,None,False,None,LOW,ENTERTAINMENT_SPORTING_EVENTS_AMUSEMENT_PARKS_AND_MUSEUMS,ENTERTAINMENT,None,ZnBBB1rQo5uyk187w43wCJJmdVGgwgfJBqx76,place,None,None
5,pQAAAxEKr3SJEe7prVQruWWpBBRbkZupnX989,None,ADDED,6.33,2024-02-06,None,"[Travel, Taxi]",22016000,None,"[{'name': 'Uber', 'type': 'merchant', 'website': 'uber.com', 'logo_url': 'https://plaid-merchant-logos.plaid.com/uber_1060.png', 'confi

In [ ]:
# partition_str = get_current_date_partition(offset_days=0)
# destination_table_id = "account_cursors_" + partition_str
# accounts_table_id = "financial_accounts"

# create_new_cursors_bq_table(
#     dataset_id="personal_finance", destination_table_id=destination_table_id, accounts_table_id=accounts_table_id
# )

In [ ]:
# for r in removed:
#     print(r)

# Run Functions

In [ ]:
# access_token = chase_access_token
# dataset_id = "personal_finance"
# table_id = "financial_accounts"

# table_ref = bq_client.dataset(dataset_id).table(table_id)

# get_cursor_df = bq_client.query(
#     f"""
# SELECT DISTINCT cursor
# FROM `{table_ref}`
# GROUP BY 1,2
# """
# ).to_dataframe()

### Delete / Create accounts table

In [ ]:
# delete_bq_table(
#     dataset_id="personal_finance",
#     table_id="financial_accounts"
# )

In [ ]:
# create_accounts_table(
#     dataset_id="personal_finance",
#     table_id="financial_accounts",
#     table_description="Stores all account data.  Field descriptions available at https://plaid.com/docs/api/accounts/#accounts-get-response-accounts-persistent-account-id"
# )

### Add Chase accounts

In [ ]:
# chase_access_token = "access-sandbox-747b2b77-8d2f-4318-9c44-635e3fc4575d"

In [ ]:
# add_accounts_to_bq(
#     access_token=chase_access_token,
#     dataset_id="personal_finance",
#     table_id="financial_accounts"
# )

### Add Schwab Accounts

In [ ]:
schwab_access_token = "access-sandbox-5f048cf7-c0fe-4811-8769-cb35b8558528"

In [ ]:
# add_accounts_to_bq(
#     access_token=schwab_access_token,
#     dataset_id="personal_finance",
#     table_id="financial_accounts"
# )

In [ ]:
pretty_print_response(get_accounts(wf_access_token))

## Delete table

# Comments

## Start

In [ ]:
# def empty_to_none(field):
#     value = os.getenv(field)
#     if value is None or len(value) == 0:
#         return None
#     return value

# host = plaid.Environment.Sandbox

# if PLAID_ENV == 'sandbox':
#     host = plaid.Environment.Sandbox

# if PLAID_ENV == 'development':
#     host = plaid.Environment.Development

# if PLAID_ENV == 'production':
#     host = plaid.Environment.Production

# configuration = plaid.Configuration(
#     host=host,
#     api_key={
#         'clientId': PLAID_CLIENT_ID,
#         'secret': PLAID_SECRET,
#         'plaidVersion': '2020-09-14'
#     }
# )

# api_client = plaid.ApiClient(configuration)
# client = plaid_api.PlaidApi(api_client)

# products = []
# for product in PLAID_PRODUCTS:
#     products.append(Products(product))

## Transactions

In [ ]:
def get_transactions(access_token):
    try:
        # Fetch transactions for the account
        request = TransactionsSyncRequest(
            access_token=access_token,
            cursor="",
            count=500
          )
        
        # request = TransactionsGetRequest(
        #     access_token=access_token,
        #     start_date=start_date,
        #     end_date=end_date,
            options=TransactionsGetRequestOptions(
                count=500
            )
        # )
        
        response = plaid_client.transactions_sync(request)
        transactions = response.to_dict()
        
        print("SUCCESS: retrieved transactions!")
        
        return transactions
        
    except plaid.ApiException as e:
        print("ERROR:", e)

In [ ]:
def create_transactions_df(transactions):
    account_ids = []
    account_owners = []
    amounts = []
    authorized_dates = []
    authorized_datetimes = []
    categories = []
    category_ids = []
    check_numbers = []
    counterparties = []
    dates = []
    datetimes = []
    currency_codes = []
    addresses = []
    cities = []
    regions = []
    postal_codes = []
    countries = []
    latitudes = []
    longitudes = []
    store_numbers = []
    merchant_entity_ids = []
    merchant_names = []
    names = []
    payment_channels = []
    reference_numbers = []
    ppd_ids = []
    payees = []
    by_order_ofs = []
    payers = []
    payment_methods = []
    payment_processors = []
    reasons = []
    pendings = []
    pending_transaction_ids = []
    personal_finance_category_confidence_levels = []
    personal_finance_category_detailed = []
    personal_finance_category_primaries = []
    transaction_codes = []
    transaction_ids = []
    transaction_types = []
    unofficial_currency_codes = []
    websites = []
    statuses = []

    for t in transactions:
        account_ids.append(t["account_id"])
        account_owners.append(t["account_owner"])
        amounts.append(t["amount"])
        authorized_dates.append(t["authorized_date"])
        authorized_datetimes.append(t["authorized_datetime"])
        categories.append(t["category"])
        category_ids.append(t["category_id"])
        check_numbers.append(t["check_number"])
        counterparties.append(t["counterparties"])
        dates.append(t["date"])
        datetimes.append(t["datetime"])
        currency_codes.append(t["iso_currency_code"])

        # transaction type
        transaction_types.append("ADDED")

        # location
        addresses.append(t["location"]["address"])
        cities.append(t["location"]["city"])
        regions.append(t["location"]["region"])
        postal_codes.append(t["location"]["postal_code"])
        countries.append(t["location"]["country"])
        latitudes.append(t["location"]["lat"])
        longitudes.append(t["location"]["lon"])
        store_numbers.append(t["location"]["store_number"])

        # merchant
        merchant_entity_ids.append(t["merchant_entity_id"])
        merchant_names.append(t["merchant_name"])
        names.append(t["name"])
        payment_channels.append(t["payment_channel"])

        # payment meta
        reference_numbers.append(t["payment_meta"]["reference_number"])
        ppd_ids.append(t["payment_meta"]["ppd_id"])
        payees.append(t["payment_meta"]["payee"])
        by_order_ofs.append(t["payment_meta"]["by_order_of"])
        payers.append(t["payment_meta"]["payer"])
        payment_methods.append(t["payment_meta"]["payment_method"])
        payment_processors.append(t["payment_meta"]["payment_processor"])
        reasons.append(t["payment_meta"]["reason"])

        pendings.append(t["pending"])
        pending_transaction_ids.append(t["pending_transaction_id"])

        # personal finance
        personal_finance_category_confidence_levels.append(t["personal_finance_category"]["confidence_level"])
        personal_finance_category_detailed.append(t["personal_finance_category"]["detailed"])
        personal_finance_category_primaries.append(t["personal_finance_category"]["primary"])

        transaction_codes.append(t["transaction_code"])
        transaction_ids.append(t["transaction_id"])
        transaction_types.append(t["transaction_type"])
        unofficial_currency_codes.append(t["unofficial_currency_code"])
        websites.append(t["website"])

    # add account data to accounts_df
    transactions_df = pd.DataFrame(
        {
            "account_id": account_ids,
            "account_owner": account_owners,
            "amount": amounts,
            "authorized_date": authorized_dates,
            "authorized_datetime": authorized_datetimes,
            "category": categories,
            "category_id": category_ids,
            "check_number": check_numbers,
            "counterparties": counterparties,
            "date": dates,
            "datetimes": datetimes,
            "currency_code": currency_codes,
            "address": addresses,
            "city": cities,
            "region": regions,
            "postal_code": postal_codes,
            "latitude": latitudes,
            "longitude": longitudes,
            "merchant_entity_id": merchant_entity_ids,
            "merchant_name": merchant_names,
            "name": names,
            "payment_channel": payment_channels,
            "reference_number": reference_numbers,
            "ppd_id": ppd_ids,
            "payee": payees,
            "by_order_of": by_order_ofs,
            "payer": payers,
            "payment_method": payment_methods,
            "payment_processor": payment_processors,
            "reason": reasons,
            "pending": pendings,
            "pending_transaction_id": pending_transaction_ids,
            "personal_finance_category_confidence_level": personal_finance_category_confidence_levels,
            "personal_finance_category_detailed": personal_finance_category_detailed,
            "personal_finance_category_primary": personal_finance_category_primaries,
            "transaction_code": transaction_codes,
            "transaction_id": transaction_ids,
            "transaction_type": transaction_types,
            "unofficial_currency_code": unofficial_currency_codes,
            "website": websites,
        }
    )

    return transactions_dfa

In [ ]:
# Define the start and end dates for the transaction query
start_date = (datetime.datetime.now() - timedelta(days=90)).date()
end_date = datetime.datetime.now().date()

In [ ]:
transactions = get_transactions(chase_access_token)

create_transactions_df(transactions)

In [ ]:
transactions_json

In [ ]:
# accounts_ids = []
# account_owners = []

# pretty_print_response(transactions_json)

In [ ]:
transactions_json["transactions"][0]["location"]

In [ ]:
# transactions_df = pd.json_normalize(transactions)

In [ ]:
# transactions_df

In [ ]:
transactions_json["transactions"]

## Test get_accounts

In [ ]:
# test = get_accounts(access_token=access_token, client=client)
# test

## Dates

## Link Tokens

In [ ]:
# def create_link_token():
#     try:
#         request = LinkTokenCreateRequest(
#             products=products,
#             client_name="Plaid Quickstart",
#             country_codes=list(map(lambda x: CountryCode(x), PLAID_COUNTRY_CODES)),
#             language='en',
#             user=LinkTokenCreateRequestUser(
#                 client_user_id=str(time.time())
#             )
#         )
#         if PLAID_REDIRECT_URI!=None:
#             request['redirect_uri']=PLAID_REDIRECT_URI
#     # create link token
#         response = client.link_token_create(request)
#         return response.to_dict()
#         # return jsonify(response.to_dict())
#     except plaid.ApiException as e:
#         return json.loads(e.body)

In [ ]:
# link_token_request = create_link_token()

In [ ]:
# link_token_request

In [ ]:
# link_token_request['link_token']

## Test BigQuery

In [ ]:
# Initialize a client
# client = bigquery.Client()

In [ ]:
# # Define your dataset ID and table ID
# dataset_id = 'personal_finance'
# table_id = 'test_table'

In [ ]:
# # Define your schema
# schema = [
#     bigquery.SchemaField("column1", "STRING"),
#     bigquery.SchemaField("column2", "INTEGER"),
#     # Add more fields as needed
# ]

# # Define your table
# table = bigquery.Table(f"{client.project}.{dataset_id}.{table_id}", schema=schema)

# # Create the table
# table = client.create_table(table)

# # Insert data into the table
# rows_to_insert = [
#     {"column1": "value1", "column2": 1},
#     {"column1": "value2", "column2": 2},
#     # Add more rows as needed
# ]

# # Insert the rows into the table
# client.insert_rows(table, rows_to_insert)

# print("Data inserted successfully!")